In [ ]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
# from keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))
with tf.device('/device:GPU:0'):

    train_dir = '../input/fer2013/train'
    val_dir = '../input/fer2013/test'
    train_datagen = ImageDataGenerator(rescale=1./255)
    val_datagen = ImageDataGenerator(rescale=1./255)

    train_generator = train_datagen.flow_from_directory(
            train_dir,
            target_size=(48,48),
            batch_size=64,
            color_mode="grayscale",
            class_mode='categorical')

    validation_generator = val_datagen.flow_from_directory(
            val_dir,
            target_size=(48,48),
            batch_size=64,
            color_mode="grayscale",
            class_mode='categorical')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam,RMSprop,SGD,Adamax
emotion_model= tf.keras.models.Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(48, 48,1)))
emotion_model.add(Conv2D(64,(3,3), padding='same', activation='relu' ))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPool2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128,(5,5), padding='same', activation='relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPool2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
    
emotion_model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPool2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(512,(3,3), padding='same', activation='relu', kernel_regularizer=regularizers.l2(0.01)))
emotion_model.add(BatchNormalization())
emotion_model.add(MaxPool2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten()) 
emotion_model.add(Dense(256,activation = 'relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Dropout(0.25))
    
emotion_model.add(Dense(512,activation = 'relu'))
emotion_model.add(BatchNormalization())
emotion_model.add(Dropout(0.25))

emotion_model.add(Dense(7, activation='softmax'))

In [ ]:
emotion_model.compile(
    optimizer = Adam(lr=0.0001), 
    loss='categorical_crossentropy', 
    metrics=['Recall']
  )

In [ ]:
emotion_model_info = emotion_model.fit(
            train_generator,
            steps_per_epoch=28709 // 64,
            epochs=60,
            validation_data=validation_generator,
            validation_steps=7178 // 64)

In [ ]:
import PIL
from PIL import Image
import os

In [ ]:
from matplotlib import image
from matplotlib import pyplot

def convertImage(dir):
    load_img_rz = np.array(Image.open(dir).resize((48,48)))
    load_img_rz = load_img_rz.reshape(-1, 48, 48, 1)
    return load_img_rz

In [ ]:
def bestprediction(imageinput):
    emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
    prediction = emotion_model.predict(imageinput)
    for r in prediction:
       for c in range(len(r)):
            r[c] = int(r[c])
    decisionarray = []
    for r in range(len(prediction)):
        for c in range(len(prediction[0])):
            if prediction[r][c] == max(prediction[r]):
                bestcol = c
                decisionarray.append(bestcol)
    finalpred = max(set(decisionarray), key=decisionarray.count)
    return emotions[finalpred]

In [ ]:
def newprediction(imageinput):
    emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
    prediction = emotion_model.predict(imageinput)
    for r in prediction:
       for c in range(len(r)):
            r[c] = int(r[c])
    decisionarray = []
    for r in range(len(prediction)):
        for c in range(len(prediction[0])):
            if prediction[r][c] == max(prediction[r]):
                bestcol = c
                decisionarray.append(bestcol)
    return decisionarray

In [ ]:
bestprediction(convertImage('../input/angerperson/rccm_anger1.jpg'))